In [ ]:
# Upgrade the library
!conda run -n fpga-env pip install pybondmachine --upgrade

In [ ]:
### IF THE PREVIOUS COMMAND DOES NOT RETURN Requirement already satisfied, RELOAD THE KERNEL !!!

In [ ]:
import os
import numpy as np

# Export the env variabes to access the alveo card

os.environ["PATH"]=os.environ["PATH"]+":"+os.environ["BONDMACHINE_DIR"]
os.environ['XILINX_HLS'] = '/tools/Xilinx/Vitis_HLS/2023.2'
os.environ['XILINX_VIVADO'] = '/tools/Xilinx/Vivado/2023.2'
os.environ['XILINX_VITIS'] = '/tools/Xilinx/Vitis/2023.2'
os.environ['PATH']=os.environ["PATH"]+":"+os.environ['XILINX_HLS']+"/bin:"+os.environ['XILINX_VIVADO']+"/bin:"+os.environ['XILINX_VITIS']+"/bin:"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['XILINX_XRT'] = '/opt/xilinx/xrt'
os.environ['LD_LIBRARY_PATH'] = '/opt/xilinx/xrt/lib'

notebook_directory = os.path.abspath(os.path.dirname((os.environ["JPY_SESSION_NAME"])))
os.chdir(notebook_directory)

In [ ]:
# Display a local  png image
from IPython.display import Image, display
display(Image(filename='bellstate.png'))

In [ ]:
# generate sample data just
# by default the firmware access a batch of 16 elements of lenght 8
n_samples = 16
length    = 8
np.random.seed(42)
X = np.zeros((n_samples, length), dtype=np.float32)
random_positions = np.random.randint(1, size=n_samples)
X[np.arange(n_samples), random_positions] = 1.0
np.save("features.npy", X)

In [ ]:
from pybondmachine.overlay.predictor import Predictor
import numpy as np
model_specs = {
    "data_type": "float32",
    "register_size": 32,
    "batch_size": 16,
    "flavor": "axist",
    "n_input": 8,
    "n_output": 8,
    "benchcore": False,
    "board": "alveo"
}
firmware_name = "bellstate.xclbin"
firmware_path = ""

In [ ]:
X_test = np.load("features.npy")
y_test = np.load("features.npy")

In [ ]:
predictor = Predictor(firmware_name, firmware_path, model_specs)

In [ ]:
# program the ALVEO CARD
predictor.load_overlay()

In [ ]:
# prepare the data to be sent to the alveo card
predictor.prepare_data(X_test, y_test)


In [ ]:
# run the computation
status, result = predictor.predict(debug=True)

In [ ]:
print(result['all_probs'][0])

In [ ]:
%%bash
source /opt/xilinx/xrt/setup.sh
xbutil examine